In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24252


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24247,2021-06-10,47,沖縄県,166,19102,0,156
24248,2021-06-11,47,沖縄県,145,19247,0,156
24249,2021-06-12,47,沖縄県,157,19404,3,159
24250,2021-06-13,47,沖縄県,104,19508,0,159
24251,2021-06-14,47,沖縄県,54,19562,4,163


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11352,8871,144
43,香川県,19092,2066,28
44,高知県,20124,1667,22
45,鳥取県,15996,466,2
46,鹿児島県,23736,3594,33


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24245,2021-06-08,47,沖縄県,159,18762,0,154,1495.0
24246,2021-06-09,47,沖縄県,174,18936,2,156,1372.0
24247,2021-06-10,47,沖縄県,166,19102,0,156,1294.0
24248,2021-06-11,47,沖縄県,145,19247,0,156,1192.0
24249,2021-06-12,47,沖縄県,157,19404,3,159,1088.0
24250,2021-06-13,47,沖縄県,104,19508,0,159,1009.0
24251,2021-06-14,47,沖縄県,54,19562,4,163,959.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24242,2021-06-05,47,沖縄県,261,18316,0,153,1685.0,0.056680,0.017961,-0.220896
24243,2021-06-06,47,沖縄県,183,18499,0,153,1597.0,-0.298851,0.002560,-0.324723
24244,2021-06-07,47,沖縄県,104,18603,1,154,1559.0,-0.431694,0.008892,-0.267606
24245,2021-06-08,47,沖縄県,159,18762,0,154,1495.0,0.528846,0.002952,-0.286996
24246,2021-06-09,47,沖縄県,174,18936,2,156,1372.0,0.094340,-0.030976,-0.414141
24247,2021-06-10,47,沖縄県,166,19102,0,156,1294.0,-0.045977,-0.012052,-0.319672
24248,2021-06-11,47,沖縄県,145,19247,0,156,1192.0,-0.126506,-0.031880,-0.412955
24249,2021-06-12,47,沖縄県,157,19404,3,159,1088.0,0.082759,-0.028155,-0.398467
24250,2021-06-13,47,沖縄県,104,19508,0,159,1009.0,-0.337580,-0.033687,-0.431694
24251,2021-06-14,47,沖縄県,54,19562,4,163,959.0,-0.480769,-0.040698,-0.480769


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["都道府県名","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,都道府県名,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
515,北海道,907.0,-0.097561,-0.065434,-0.496599
1031,青森県,22.0,-1.000000,-0.307937,-1.000000
1547,岩手県,89.0,-0.100000,0.170805,0.125000
2063,宮城県,59.0,-0.142857,0.422052,2.000000
2579,秋田県,3.0,-1.000000,NaN,NaN


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12384,5148,109,93.0,-0.777778,0.079762,-0.555556
1,京都府,13416,16357,230,245.0,-0.608696,0.351119,-0.100000
2,佐賀県,21156,2539,23,11.0,0.000000,NaN,inf
3,兵庫県,14448,40534,1259,309.0,-0.086957,0.153632,-0.045455
4,北海道,516,40546,1301,907.0,-0.097561,-0.065434,-0.496599


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12384,5148,109,93.0,-0.777778,0.079762,-0.555556,1813859
1,京都府,13416,16357,230,245.0,-0.608696,0.351119,-0.100000,2545899
2,佐賀県,21156,2539,23,11.0,0.000000,NaN,inf,823810
3,兵庫県,14448,40534,1259,309.0,-0.086957,0.153632,-0.045455,5549568
4,北海道,516,40546,1301,907.0,-0.097561,-0.065434,-0.496599,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,516,40546,1301,907.0,-0.097561,-0.065434,-0.496599,5267762,76.970068,769.700681,17.217938
41,青森県,1032,2448,30,22.0,-1.000000,-0.307937,-1.000000,1275783,19.188216,191.882162,1.724431
19,岩手県,1548,1599,46,89.0,-0.100000,0.170805,0.125000,1235517,12.941951,129.419506,7.203462
11,宮城県,2064,9036,84,59.0,-0.142857,0.422052,2.000000,2292385,39.417463,394.174626,2.573739
36,秋田県,2580,763,20,3.0,-1.000000,NaN,NaN,985416,7.742923,77.429228,0.304440
15,山形県,3096,2011,46,12.0,0.000000,NaN,1.000000,1082296,18.580869,185.808688,1.108754
35,福島県,3612,4718,158,62.0,-0.692308,0.355128,-0.692308,1881981,25.069329,250.693285,3.294401
38,茨城県,4128,10147,157,198.0,-0.500000,0.475000,-0.500000,2921436,34.732919,347.329190,6.777489
27,栃木県,4644,6688,80,179.0,0.888889,0.104592,-0.190476,1965516,34.026688,340.266882,9.107023
37,群馬県,5160,7983,149,55.0,-0.250000,0.060215,-0.727273,1969439,40.534386,405.343857,2.792673


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16445.574468
std       30104.802165
min         466.000000
25%        2493.500000
50%        4936.000000
75%       10701.000000
max      166578.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6708,166578,2159,2663.0,-0.312500,0.023324,-0.110638,13834925,120.403978,1204.039776,19.248388
1,大阪府,13932,101831,2540,904.0,-0.406250,0.087547,-0.208333,8849635,115.068023,1150.680226,10.215111
2,神奈川県,7224,64334,924,1347.0,-0.170588,-0.013956,-0.184971,9209442,69.856567,698.565668,14.626293
3,愛知県,11868,49927,895,1008.0,-0.549020,-0.017868,-0.525773,7575530,65.905620,659.056198,13.306000
4,埼玉県,5676,45078,819,590.0,-0.369231,0.058874,-0.068182,7390054,60.998201,609.982011,7.983704
5,北海道,516,40546,1301,907.0,-0.097561,-0.065434,-0.496599,5267762,76.970068,769.700681,17.217938
6,兵庫県,14448,40534,1259,309.0,-0.086957,0.153632,-0.045455,5549568,73.039920,730.399195,5.568001
7,千葉県,6192,38480,694,677.0,-0.133333,0.066403,0.338235,6319772,60.888273,608.882726,10.712412
8,福岡県,20640,34996,491,329.0,-0.361111,0.071645,-0.148148,5129841,68.220438,682.204380,6.413454
9,沖縄県,24252,19562,163,959.0,-0.480769,-0.040698,-0.480769,1481547,132.037661,1320.376606,64.729637


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      39.652623
std       28.649189
min        7.742923
25%       22.107653
50%       30.124328
75%       51.752261
max      132.037661
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24252,19562,163,959.0,-0.480769,-0.040698,-0.480769,1481547,132.037661,1320.376606,64.729637
16,山梨県,9804,1848,20,164.0,-0.222222,0.254642,-0.416667,826579,22.357210,223.572097,19.840814
26,東京都,6708,166578,2159,2663.0,-0.312500,0.023324,-0.110638,13834925,120.403978,1204.039776,19.248388
4,北海道,516,40546,1301,907.0,-0.097561,-0.065434,-0.496599,5267762,76.970068,769.700681,17.217938
32,神奈川県,7224,64334,924,1347.0,-0.170588,-0.013956,-0.184971,9209442,69.856567,698.565668,14.626293
24,愛知県,11868,49927,895,1008.0,-0.549020,-0.017868,-0.525773,7575530,65.905620,659.056198,13.306000
5,千葉県,6192,38480,694,677.0,-0.133333,0.066403,0.338235,6319772,60.888273,608.882726,10.712412
9,大阪府,13932,101831,2540,904.0,-0.406250,0.087547,-0.208333,8849635,115.068023,1150.680226,10.215111
1,京都府,13416,16357,230,245.0,-0.608696,0.351119,-0.100000,2545899,64.248425,642.484246,9.623320
27,栃木県,4644,6688,80,179.0,0.888889,0.104592,-0.190476,1965516,34.026688,340.266882,9.107023


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    37.000000
mean      0.145333
std       0.212612
min      -0.307937
25%      -0.013956
50%       0.087547
75%       0.259524
max       0.721088
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
31,石川県,8772,3880,112,37.0,-0.166667,0.721088,0.250000,1139612,34.046676,340.466755,3.246719
13,富山県,8256,1964,37,38.0,0.000000,0.587755,0.250000,1055999,18.598502,185.985025,3.598488
38,茨城県,4128,10147,157,198.0,-0.500000,0.475000,-0.500000,2921436,34.732919,347.329190,6.777489
11,宮城県,2064,9036,84,59.0,-0.142857,0.422052,2.000000,2292385,39.417463,394.174626,2.573739
14,山口県,18060,3069,71,69.0,2.000000,0.364659,-0.500000,1369882,22.403389,224.033895,5.036930
6,和歌山県,15480,2652,48,14.0,-1.000000,0.357143,-1.000000,954258,27.791226,277.912263,1.467108
35,福島県,3612,4718,158,62.0,-0.692308,0.355128,-0.692308,1881981,25.069329,250.693285,3.294401
1,京都府,13416,16357,230,245.0,-0.608696,0.351119,-0.100000,2545899,64.248425,642.484246,9.623320
25,新潟県,7740,3364,46,65.0,-0.600000,0.344596,0.000000,2236042,15.044440,150.444401,2.906922
43,香川県,19092,2066,28,28.0,0.000000,0.259524,0.000000,981280,21.054133,210.541334,2.853416


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
41,青森県,1032,2448,30,22.0,-1.000000,-0.307937,-1.000000,1275783,19.188216,191.882162,1.724431
18,岡山県,17028,7551,120,63.0,-0.833333,-0.165210,-0.916667,1903627,39.666384,396.663842,3.309472
4,北海道,516,40546,1301,907.0,-0.097561,-0.065434,-0.496599,5267762,76.970068,769.700681,17.217938
30,熊本県,22188,6404,110,50.0,-1.000000,-0.063579,-1.000000,1769880,36.183244,361.832441,2.825050
28,沖縄県,24252,19562,163,959.0,-0.480769,-0.040698,-0.480769,1481547,132.037661,1320.376606,64.729637
17,岐阜県,10836,9096,180,143.0,0.100000,-0.040390,-0.560000,2032490,44.752988,447.529877,7.035705
44,高知県,20124,1667,22,57.0,-0.333333,-0.033916,-0.400000,709230,23.504364,235.043639,8.036885
42,静岡県,11352,8871,144,210.0,-0.666667,-0.024253,-0.565217,3708556,23.920361,239.203615,5.662581
24,愛知県,11868,49927,895,1008.0,-0.549020,-0.017868,-0.525773,7575530,65.905620,659.056198,13.306000
32,神奈川県,7224,64334,924,1347.0,-0.170588,-0.013956,-0.184971,9209442,69.856567,698.565668,14.626293


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,10))
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
#plt.xticks(rotation="70",fontsize=9)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
#plt.bar(all_jp_desc["都道府県名"], all_jp_desc["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")
#plt.title("各地の感染者数", y=0.9)
#plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])
#plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,3) # (rows, columns, panel number)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.title("直近1週間の10万人当たりの新規感染者数")
# plt.text(0,chart_data["都道府県名"][-1],"{}時点".format(latest_date))
#plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

### 新規感染者数の増加速度（直近１週間の前日比平均）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
#chart_data = all_jp.sort_values("直近1週間の新規感染者数前日比平均",na_position="first")

plt.yticks(fontsize=8)
plt.grid(axis="x")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
#plt.text(chart_data["直近1週間の新規感染者数前日比平均"].max(),-1,"※{}時点".format(latest_date),fontsize=9,ha="right")

#fig.text(0,0,"※{}時点".format(latest_date),fontsize=9)
fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()


<IPython.core.display.Javascript object>

In [22]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [23]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.  21230.  41994.  62758.  83522. 104286. 125050. 145814. 166578.]


## ヒストグラム

In [24]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([38.,  2.,  5.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([10., 23.,  3.,  8.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [26]:
pref[pref["都道府県名"]=="和歌山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
15470,2021-06-05,30,和歌山県,2,2635,0,45,23.0,inf,NaN,-0.882353
15471,2021-06-06,30,和歌山県,2,2637,0,45,20.0,0.000000,NaN,-0.600000
15472,2021-06-07,30,和歌山県,1,2638,0,45,20.0,-0.500000,NaN,0.000000
15473,2021-06-08,30,和歌山県,1,2639,0,45,18.0,0.000000,NaN,-0.666667
15474,2021-06-09,30,和歌山県,2,2641,2,47,12.0,1.000000,NaN,-0.750000
15475,2021-06-10,30,和歌山県,6,2647,0,47,14.0,2.000000,NaN,0.500000
15476,2021-06-11,30,和歌山県,1,2648,0,47,15.0,-0.833333,NaN,inf
15477,2021-06-12,30,和歌山県,3,2651,0,47,16.0,2.000000,0.523810,0.500000
15478,2021-06-13,30,和歌山県,1,2652,0,47,15.0,-0.666667,0.428571,-0.500000
15479,2021-06-14,30,和歌山県,0,2652,1,48,14.0,-1.000000,0.357143,-1.000000
